In [3]:
from fastai2.collab import *
from fastai2.tabular.all import *
import zipfile

In [21]:
#Dataset imported from ('https://grouplens.org/datasets/movielens/25m/')

In [39]:
with zipfile.ZipFile('ml-25m.zip', 'r') as f:
    f.extractall()

In [4]:
path = Path.cwd()/'ml-25m'

In [5]:
ratings = pd.read_csv(path/'ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [6]:
movies = pd.read_csv(path/'movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings = ratings.merge(movies)
ratings.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [13]:
dls= CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

MemoryError: 

In [ ]:
dls.classes

In [ ]:
n_users = len(dls['userId'])
n_movies = len(dls['movieId'])
n_factors = 5

In [ ]:
x,y = dls.one_batch()
x.shape

In [ ]:
class CollabNN(Module):
    def __init__(self, user_sz, movie_sz, y_range(0,5.5), n_act=100):
        self.user_factors = Embedding(*user_sz)
        self.movie_factors = Embedding(*movie_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + movie_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act,1))
        self.y_range = y_range
        
    def forward(self, x):
        embs = self.user_factors(x[:,0]),self.movie_factors(x[:,1])
        x = self.layers(torch.cat(embs, dim=1)) #each embeds has shape (number,optimum emb_sz). Concat at dim=1 means join at dimension of n_users and n_movies.
        return sigmoid_range(x, *self.y_range)    

In [ ]:
embs = get_emb_sz(dls)
embs

In [ ]:
model = CollabNN(*embs)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)